In [1]:
import os, shutil
import pandas as pd
        
def Columns_Remake():
    Dataset_Remake = pd.read_csv(os.path.join("UK Accident Dataset", "Accident_Information.csv"), encoding="latin1", low_memory=False).copy()
    Dataset_Remake.columns = [col.replace("_", " ") for col in Dataset_Remake.columns]
    if "Date" in Dataset_Remake.columns:
        Dataset_Remake["Date"] = (
            pd.to_datetime(Dataset_Remake["Date"], format="%Y-%m-%d", errors="coerce")
        ).dt.strftime("%d-%m-%Y")
    elif "Time" in Dataset_Remake.columns:
        Dataset_Remake["Time"] = (
            pd.to_datetime(Dataset_Remake["Time"], format="%H:%M", errors="coerce")
        ).dt.strftime("%H:%M:%S")
        
    Dataset_Remake.to_csv(
        os.path.join("UK Accident Dataset", "Accident_Information_Remake.csv"),
        index=False,
    )        

def Remake():
    os.makedirs("UK Accident Dataset", exist_ok=True)
    os.makedirs("Model", exist_ok=True)     
    if "Accident_Information.csv" in os.listdir():
        shutil.move("Accident_Information.csv", "UK Accident Dataset")
        Columns_Remake()
    return pd.read_csv(os.path.join("UK Accident Dataset", "Accident_Information_Remake.csv"), encoding="latin1", low_memory=False).copy()

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Tắt tất cả warning và info logs

import pandas as pd
import numpy as np
from Remake_Dataset import Remake
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.utils.np_utils import to_categorical
import tensorflow as tf

# Thiết lập style cho đồ thị - sửa phần này
plt.style.use('default')  # Thay đổi từ 'seaborn' thành 'default'
sns.set_theme()  # Cách thiết lập mới cho seaborn

def load_and_prepare_data():
    print("1. Đang tải và chuẩn bị dữ liệu...")
    df = Remake()
    
    # Chọn features quan trọng
    features = [
        'Road Type', 'Speed limit', 'Light Conditions', 
        'Weather Conditions', 'Road Surface Conditions', 
        'Urban or Rural Area', 'Junction Detail', 
        'Junction Control', 'Pedestrian Crossing-Human Control',
        'Pedestrian Crossing-Physical Facilities'
    ]
    target = 'Accident Severity'
    
    # Xử lý missing values
    for col in features:
        df[col].fillna(df[col].mode()[0], inplace=True)
    
    return df, features, target

def encode_features(df, features, target):
    print("2. Đang mã hóa features...")
    X = df[features].copy()
    encoders = {}
    
    # Mã hóa từng feature
    for column in X.columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column].astype(str))
        encoders[column] = le
    
    # Mã hóa target
    le_target = LabelEncoder()
    y = le_target.fit_transform(df[target])
    
    return X, y, encoders, le_target

def train_random_forest(X, y):
    print("3. Huấn luyện mô hình Random Forest...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Khởi tạo và huấn luyện mô hình
    rf_model = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    )
    
    # Cross-validation
    cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
    print(f"\nĐộ chính xác cross-validation: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    
    # Huấn luyện mô hình cuối cùng
    rf_model.fit(X_train, y_train)
    
    return rf_model, X_train, X_test, y_train, y_test

def evaluate_model(model, X_test, y_test, features):
    print("\n4. Đánh giá mô hình...")
    y_pred = model.predict(X_test)
    
    # Tính độ chính xác
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nĐộ chính xác của mô hình: {accuracy:.4f}")
    
    # In báo cáo phân loại với zero_division=1
    print("\nBáo cáo phân loại chi tiết:")
    print(classification_report(y_test, y_pred, zero_division=1))
    
    # Vẽ confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig('Model/confusion_matrix.png')
    plt.close()
    
    # Vẽ feature importance
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='importance', y='feature', data=feature_importance)
    plt.title('Feature Importance')
    plt.savefig('Model/feature_importance.png')
    plt.close()
    
    return feature_importance

def train_neural_network(X, y):
    print("\n5. Huấn luyện Neural Network để so sánh...")
    try:
        # Chuẩn hóa dữ liệu
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Chia dữ liệu
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        
        # Chuyển đổi dữ liệu thành numpy array
        X_train = np.array(X_train, dtype=np.float32)
        X_test = np.array(X_test, dtype=np.float32)
        
        # One-hot encoding cho target
        num_classes = len(np.unique(y))
        y_train_cat = np.eye(num_classes)[y_train]
        y_test_cat = np.eye(num_classes)[y_test]
        
        # Xây dựng model với các layer đơn giản hơn
        model = Sequential()
        model.add(Dense(32, activation='relu', input_dim=X.shape[1]))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        
        # Compile với các tham số cơ bản
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Huấn luyện với batch size nhỏ hơn
        history = model.fit(
            X_train, y_train_cat,
            validation_data=(X_test, y_test_cat),
            epochs=20,
            batch_size=32,
            verbose=1
        )
        
        # Vẽ learning curves
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('Model/neural_network_performance.png')
        plt.close()
        
        return model, scaler
        
    except Exception as e:
        print(f"\nLỗi khi huấn luyện Neural Network: {str(e)}")
        print("Bỏ qua phần Neural Network và tiếp tục với Random Forest...")
        return None, None

def main():
    # Tải và chuẩn bị dữ liệu
    df, features, target = load_and_prepare_data()
    X, y, encoders, le_target = encode_features(df, features, target)
    
    # Huấn luyện và đánh giá Random Forest
    rf_model, X_train, X_test, y_train, y_test = train_random_forest(X, y)
    feature_importance = evaluate_model(rf_model, X_test, y_test, features)
    
    # Huấn luyện Neural Network
    nn_model, scaler = train_neural_network(X, y)
    
    # Lưu các model và encoder
    print("\n6. Lưu các model và encoder...")
    joblib.dump(rf_model, 'Model/random_forest_model.joblib')
    joblib.dump(encoders, 'Model/feature_encoders.joblib')
    joblib.dump(le_target, 'Model/target_encoder.joblib')
    
    if nn_model is not None:
        joblib.dump(scaler, 'Model/scaler.joblib')
        nn_model.save('Model/neural_network_model.h5')
    
    print("\nQuá trình huấn luyện hoàn tất!")
    print("Các file đã được lưu trong thư mục 'Model':")
    print("- Model Random Forest: random_forest_model.joblib")
    print("- Encoders: feature_encoders.joblib")
    print("- Target Encoder: target_encoder.joblib")
    if nn_model is not None:
        print("- Neural Network: neural_network_model.h5")
        print("- Scaler: scaler.joblib")
        print("- Đồ thị Neural Network Performance: neural_network_performance.png")
    print("- Đồ thị Confusion Matrix: confusion_matrix.png")
    print("- Đồ thị Feature Importance: feature_importance.png")

if __name__ == "__main__":
    main()